In [10]:
import customtkinter as ctk
from sklearn.cluster import KMeans
import numpy as np
import threading
import time

In [11]:
# Set appearance and theme
ctk.set_appearance_mode("Dark")
ctk.set_default_color_theme("blue")

In [12]:

class ExamManagementSystem(ctk.CTk):
    def generate_seating_plan(self):
        # Display loading screen
        loading_window = LoadingWindow(self)
        loading_window.start_loading()

        # Run the plan generation in a separate thread
        threading.Thread(target=self._generate_seating_plan, args=(loading_window,)).start()

    def _generate_seating_plan(self, loading_window):
        try:
            # Get input values
            num_students = int(self.num_students_entry.get())
            num_rooms = int(self.num_rooms_entry.get())

            # Generate random student data
            np.random.seed(0)
            student_data = np.random.randint(0, len(self.domains), size=(num_students, 1))

            # Define the k-means clustering model
            kmeans = KMeans(n_clusters=num_rooms)

            # Fit the model to the student data
            kmeans.fit(student_data)

            # Get the cluster labels for each student
            labels = kmeans.labels_

            # Close the loading window
            loading_window.stop_loading()

            # Display the seating plan in a new window
            SeatingPlanWindow(self, num_rooms, labels, student_data, self.domains, self.faculties)

        except ValueError:
            loading_window.stop_loading()
            self.seating_plan_text.configure(state="normal")
            self.seating_plan_text.delete(1.0, ctk.END)
            self.seating_plan_text.insert(ctk.END, "Invalid input. Please enter integers only.")
            self.seating_plan_text.configure(state="disabled")

    def quit_me(self):
        self.quit()
        self.destroy()

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        # Setting Window Title And Size
        self.title("Automated Exam Management System")
        self.geometry("1000x500")

        # Making All Components (Labels/Entry/Button/Text Areas)
        self.titleLabel = ctk.CTkLabel(self, text="AUTOMATED EXAM MANAGEMENT SYSTEM", font=("Arial", 40, "bold", "underline"), text_color="White")
        self.titleLabel.place(relx=0.5, rely=0.1, anchor=ctk.N)

        self.num_students_label = ctk.CTkLabel(self, width=150, text="Number of Students:", text_color="White", font=("Arial", 20))
        self.num_students_label.place(x=100, rely=0.3)

        self.num_students_entry = ctk.CTkEntry(self, width=150, font=("Arial", 20))
        self.num_students_entry.place(x=300, rely=0.3)

        self.num_rooms_label = ctk.CTkLabel(self, width=150, text="Number of Rooms:", text_color="White", font=("Arial", 20))
        self.num_rooms_label.place(x=600, rely=0.3)

        self.num_rooms_entry = ctk.CTkEntry(self, width=150, font=("Arial", 20))
        self.num_rooms_entry.place(x=800, rely=0.3)

        self.generate_button = ctk.CTkButton(self, width=250, text="Generate Seating Plan", command=self.generate_seating_plan, font=("Arial", 30))
        self.generate_button.place(x=375, rely=0.5)

        self.seating_plan_text = ctk.CTkTextbox(self, width=800, height=100, font=("Arial", 20), state="disabled")
        self.seating_plan_text.place(x=100, rely=0.6)

        self.faculty_allocation_text = ctk.CTkTextbox(self, width=800, height=100, font=("Arial", 20), state="disabled")
        self.faculty_allocation_text.place(x=100, rely=0.75)

        # Domain and faculty mappings
        self.domains = ['CS', 'AI', 'BBA', 'ME', 'Math', 'EE']
        self.faculties = ['Faculty 1', 'Faculty 2', 'Faculty 3', 'Faculty 4', 'Faculty 5', 'Faculty 6']

class LoadingWindow:
    def __init__(self, parent):
        self.parent = parent
        self.loading_window = ctk.CTkToplevel(parent)
        self.loading_window.geometry("300x100")
        self.loading_label = ctk.CTkLabel(self.loading_window, text="Generating seating plan...", font=("Arial", 20))
        self.loading_label.pack(expand=True)

    def start_loading(self):
        self.loading_window.update()
        self.loading_window.deiconify()

    def stop_loading(self):
        self.loading_window.withdraw()

In [13]:
class SeatingPlanWindow(ctk.CTkToplevel):
    def __init__(self, parent, num_rooms, labels, student_data, domains, faculties):
        super().__init__(parent)
        self.title("Seating Plan and Faculty Allocation")
        self.geometry("800x600")

        seating_plan_text = ctk.CTkTextbox(self, width=700, height=200, font=("Arial", 20))
        seating_plan_text.pack(pady=20)
        seating_plan_text.configure(state="normal")
        seating_plan_text.delete(1.0, ctk.END)

        for i in range(num_rooms):
            room_students = np.where(labels == i)[0]
            seating_plan_text.insert(ctk.END, f"Room {i+1}:\n")
            for student in room_students:
                seating_plan_text.insert(ctk.END, f"Student {student+1} - {domains[student_data[student, 0]]}\n")
            seating_plan_text.insert(ctk.END, "\n")
        seating_plan_text.configure(state="disabled")

        faculty_allocation_text = ctk.CTkTextbox(self, width=700, height=200, font=("Arial", 20))
        faculty_allocation_text.pack(pady=20)
        faculty_allocation_text.configure(state="normal")
        faculty_allocation_text.delete(1.0, ctk.END)

        for i in range(num_rooms):
            faculty_allocation_text.insert(ctk.END, f"Room {i+1} - {faculties[i % len(faculties)]}\n")
        faculty_allocation_text.configure(state="disabled")


In [14]:
# Running The App (GUI)
if __name__ == "__main__":
    app = ExamManagementSystem()
    app.protocol("WM_DELETE_WINDOW", app.quit_me)
    app.mainloop()


D:\New folder\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
D:\New folder\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
D:\New folder\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
D:\New folder\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can